In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\kidney-disease-classification'

In [ ]:
#credentials

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    train_data_path: Path
    all_params: dict
    mlflow_tracking_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        evaluation_config = ModelEvaluationConfig(
            model_path="artifacts/training/model.h5",
            train_data_path="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            mlflow_tracking_uri="https://dagshub.com/arshpatel213/kidney-disease-classification.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config
    
    
    
    
    


In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def train_valid_generator(self):

        ## data generator copied from keras documentation
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
    

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.train_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(model_path: str) -> tf.keras.Model:
        model = tf.keras.models.load_model(model_path)
        return model
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self.train_valid_generator()
        self.results = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores={"loss": self.results[0], "accuracy": self.results[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_tracking_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.results[0], "accuracy": self.results[1]})
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="KidneyDiseaseModel")
            else:
                mlflow.keras.log_model(self.model, "model")
                


In [13]:
try:
    config=ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(eval_config)
    model_eval.evaluation()
    model_eval.log_into_mlflow()
    
except Exception as e:
    print(f"Error occurred: {e}")
    raise e

YAML content loaded: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharin', 'local_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/updated_model.h5'}, 'training': {'root_dir': 'artifacts/training', 'model_checkpoint': 'artifacts/training/model.h5'}}
YAML content loaded: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'EPOCHS': 200, 'LEARNING_RATE': 0.02}
[2025-08-22 22:13:21,304] INFO in common: Created directory: artifacts
Found 93 images belonging to 2 classes.
3/3 [==============================] - 5s 2s/step - loss: 0.0281 - accuracy: 0.9892


2025/08/22 22:13:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-22 22:13:29,952] WARNING in save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: C:\Users\ARSH\AppData\Local\Temp\tmp5zi4b7xo\model\data\model\assets
[2025-08-22 22:13:30,390] INFO in builder_impl: Assets written to: C:\Users\ARSH\AppData\Local\Temp\tmp5zi4b7xo\model\data\model\assets


c:\Users\ARSH\Desktop\MLOps\kidney-disease-classification\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'KidneyDiseaseModel' already exists. Creating a new version of this model...
2025/08/22 22:15:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: KidneyDiseaseModel, version 3
Created version '3' of model 'KidneyDiseaseModel'.
